In [ ]:
%%sql -r dataframe_1
select
  metadata$filename,
  $1
from '@"TEST"."TEST_SCHEMA"."INTERNAL_STAGE"/json/bronze_test_array.json' 
;


In [ ]:
%%sql -r dataframe_2
use database TEST;
use schema TEST_SCHEMA;

create or replace file format ff_json
  type = json;


In [ ]:
%%sql -r dataframe_3
select
  $1 as json_data
from '@"TEST"."TEST_SCHEMA"."INTERNAL_STAGE"/json/bronze_test_array.json' 
(file_format => ff_json);


In [ ]:
%%sql -r dataframe_4
select
  f.value
from '@"TEST"."TEST_SCHEMA"."INTERNAL_STAGE"/json/bronze_test_array.json' 
(file_format => ff_json),
lateral flatten(input => $1) f;


In [ ]:
%%sql -r dataframe_5
with src as (
  select $1 as json_data
  from @"TEST"."TEST_SCHEMA"."INTERNAL_STAGE"/json/bronze_test_array.json
  (file_format => ff_json)
)
select
  f.value:record_id::number   as record_id,
  f.value:student_id::string  as student_id,
  f.value:score::number       as score,
  f.value:optional_null       as optional_null,
  f.value:comment::string     as comment,
  f.value:extra_col::string   as extra_col
from src,
     lateral flatten(input => src.json_data) f
order by record_id;


In [ ]:
%%sql -r dataframe_6
create or replace table TEST.TEST_SCHEMA.raw_json_variant (
  payload   variant,
  load_ts   timestamp_ntz default current_timestamp()
);


In [ ]:
%%sql -r dataframe_7
insert into raw_json_variant (payload)
select
  f.value
from @"TEST"."TEST_SCHEMA"."INTERNAL_STAGE"/json/bronze_test_array.json
     (file_format => ff_json),
     lateral flatten(input => $1) f;


In [ ]:
%%sql -r dataframe_8
select
  payload
from raw_json_variant;


In [ ]:
%%sql -r dataframe_9
create or replace view v_student_score as
select
  payload:record_id::number  as record_id,
  payload:student_id::string as student_id,
  payload:score::number      as score
from raw_json_variant;


In [ ]:
%%sql -r dataframe_10
select * from v_student_score;


In [ ]:
%%sql -r dataframe_11
insert into raw_json_variant(payload)
select f.value
from '@"TEST"."TEST_SCHEMA"."INTERNAL_STAGE"/json/bronze_test_multiline.json' (file_format => ff_json),
     lateral flatten(input => $1) f;

In [ ]:
%%sql -r dataframe_12
select
  payload:record_id::number as record_id,
  payload:event_time::string as event_time_text,
  payload:nested:a::number as nested_a,
  payload:nested:b as nested_b,
  payload:arr[0]::number as arr_0,
  payload:arr[1] as arr_1,
  payload:arr[2]::number as arr_2
from raw_json_variant
order by record_id;


In [ ]:
%%sql -r dataframe_13
select * from raw_json_variant

In [ ]:
%%sql -r dataframe_14
select
  -- 共通キー
  payload:record_id::number        as record_id,
  payload:student_id::string       as student_id,
  payload:score::number            as score,

  -- オプション項目
  payload:optional_null            as optional_null,
  payload:comment::string          as comment,
  payload:extra_col::string        as extra_col,

  -- 日時（文字列 → TIMESTAMP_TZ）
  try_to_timestamp_tz(payload:event_time::string) as event_time,

  -- ネストオブジェクト
  payload:nested:a::number         as nested_a,
  payload:nested:b                as nested_b,

  -- 配列（固定長として列化）
  payload:arr[0]::number           as arr_0,
  payload:arr[1]                  as arr_1,
  payload:arr[2]::number           as arr_2,

  -- メタ
  load_ts
from raw_json_variant
order by record_id;


In [ ]:
%%sql -r dataframe_15
create or replace dynamic table TEST.TEST_SCHEMA.dt_student_json
  target_lag = '5 minutes'
  warehouse = COMPUTE_WH
  refresh_mode = incremental
as
select
  payload:record_id::number        as record_id,
  payload:student_id::string       as student_id,
  payload:score::number            as score,
  payload:optional_null            as optional_null,
  payload:comment::string          as comment,
  payload:extra_col::string        as extra_col,
  try_to_timestamp_tz(payload:event_time::string) as event_time,
  payload:nested:a::number         as nested_a,
  payload:nested:b                as nested_b,
  payload:arr[0]::number           as arr_0,
  payload:arr[1]                  as arr_1,
  payload:arr[2]::number           as arr_2,
  load_ts
from raw_json_variant;


In [ ]:
%%sql -r dataframe_16
select * from dt_student_json
